In [ ]:
# %% Calculus 2 - Section 14.94
#    Code challenge: numerical double integrals

# This code pertains to a calculus course provided by Mike X. Cohen on Udemy:
#   > https://www.udemy.com/course/pycalc2_x
# The code in this repository is developed to solve the exercises provided along
# the course, and it has been written partially indepentently and partially
# from the code developed by the course instructor.


In [1]:
import numpy                as np
import sympy                as sym
import matplotlib.pyplot    as plt
import matplotlib.colors    as mcolors
import scipy.integrate      as spi
import math
import mpmath
import plotly.graph_objects as go
import sympy.stats

from scipy.signal                     import find_peaks
from scipy                            import stats
from IPython.display                  import display,Math
from google.colab                     import files
from IPython.display                  import Audio
from scipy.io                         import wavfile
from mpl_toolkits.mplot3d             import Axes3D
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg')

import matplotlib.animation as animation
from matplotlib import rc
rc('animation', html='jshtml')


In [ ]:
# %% Exercise 1
#    Reproduce the function in the video with sympy, compute double integral and
#    the definite integral with bounds t=[0,1] and s=[0,2]

# Variables
t = sym.symbols('t')
s = sym.symbols('s')

# Function
f        = sym.sin(sym.sqrt(t**2 + s**2)) + t/10
s_bounds = [0,2]
t_bounds = [0,1]

# Indefinite and definite integral
F_ts    = sym.integrate(f, (t,s))
def_int = sym.integrate(f, (t,t_bounds[0],t_bounds[1]), (s,s_bounds[0],s_bounds[1]))

display(Math('f(t,s) = %s' %sym.latex(f))), print()
display(Math('\\int\\int f(t,s) \\,dt \\,ds= %s' %sym.latex(F_ts))), print()
display(Math('%s = %s' %(sym.latex(sym.Integral(f,(t,t_bounds[0],t_bounds[1]),(s,s_bounds[0],s_bounds[1]))),sym.latex(def_int))))


In [ ]:
# %% Exercise 2
#    Recode the function in numpy, evaluate it over a grid of range [-5,5], plot
#    and also visualise some isometric courves using plt.contourf()

# Define the function in numpy
def f_xy(x,y):
    return np.sin(np.sqrt(x**2 + y**2)) + x/10

# Inputs range
a,b = -5,5

# Meshgrid (square) and resolution
x   = np.linspace(a,b,50)
y   = np.linspace(a,b,50)
dx  = x[1]-x[0]
X,Y = np.meshgrid(x,y)

# Function surface
function_surf = f_xy(X,Y)

# Plot
phi = (1 + np.sqrt(5)) / 2
fig,ax = plt.subplots(1,figsize=(phi*5,5))

c = ax.contourf(X,Y,function_surf,levels=50,cmap='jet',vmin=-1.5,vmax=1.2)

fig.colorbar(c,ax=ax)
ax.set(xlabel='x',ylabel='y')
ax.set_title(r'$f(x,y) = \sin\left(\sqrt{x^2+y^2}\right) + x/10$')

plt.savefig('fig9_codechallenge_94_exercise_2.png')
plt.show()
files.download('fig9_codechallenge_94_exercise_2.png')


In [ ]:
# %% Exercise 3
#    Plot all the functions (functions, double integral, partial integrals);
#    approximate the functions with numpy since simpy doesn't work so well (use
#    a double for loop, integrate each var keeping the other fixed)

# Preallocate partial integral matrices
integral_x  = np.zeros_like(X)
integral_y  = np.zeros_like(Y)
integral_xy = np.zeros_like(Y)
integral_yx = np.zeros_like(Y)

# Constant of integration (the second for exercise 6)
C = 0
C = f_xy(-5,0)

# Get the integrals by cumulatively summing the function values
for x_i in range(len(x)):
    for y_j in range(len(y)):

        # Integrate over x, keep y fixed
        integral_x[y_j,x_i] = np.sum(function_surf[y_j,:x_i]) * dx + C

        # Integrate over y, keep x fixed
        integral_y[y_j,x_i] = np.sum(function_surf[:y_j,x_i]) * dx + C

        # Double integrals (integrate the partial integrals; note that we double
        # scale by dx because both x and y have the same resolution, and we are
        # integrating over both dx and dy)
        integral_xy[y_j,x_i] = np.sum(integral_x[:y_j,x_i]) * dx
        integral_yx[y_j,x_i] = np.sum(integral_y[y_j,:x_i]) * dx


# The double integrals are the same
print('The double integrals are equal (max deviation):')
print(np.max(np.round(integral_xy-integral_yx,10)))

# Plot
phi = (1 + np.sqrt(5)) / 2
fig,axs = plt.subplots(2,2,figsize=(phi*7,7))

c1 = axs[0,0].contourf(X,Y,function_surf,levels=50,cmap='jet',vmin=-1.5,vmax=1.2)
fig.colorbar(c1, ax=axs[0,0])
axs[0,0].set_title('The function')

c2 = axs[0,1].contourf(X,Y,integral_xy,levels=50,cmap='jet',vmin=-27,vmax=0)
fig.colorbar(c2, ax=axs[0,1])
axs[0,1].set_title(r'$\int\int f(x,y) \,dx\,dy$')


c3 = axs[1,0].contourf(X, Y, integral_x,levels=50,cmap='jet',vmin=-7.2,vmax=2.2)
fig.colorbar(c3, ax=axs[1,0])
axs[1,0].set_title(r'$\int f(x,y) \,dx$')

c4 = axs[1,1].contourf(X, Y, integral_y,levels=50,cmap='jet',vmin=-11,vmax=3)
fig.colorbar(c4, ax=axs[1,1])
axs[1,1].set_title(r'$\int f(x,y) \,dy$')

plt.tight_layout()

plt.savefig('fig10_codechallenge_94_exercise_3.png')
plt.show()
files.download('fig10_codechallenge_94_exercise_3.png')


In [ ]:
# %% Exercise 4
#    Repeat exercise 3 but use scipy .quad() and .dbquad(); visualise also as in
#    exercise 3; mind that .dbquad() is a pain in the ass to use and requires
#    swapping the variables

# Preallocate partial integral matrices
integral_xs  = np.zeros_like(X)
integral_ys  = np.zeros_like(Y)
integral_xys = np.zeros_like(X)

# Use "reverse" order for .dbquad()
def f_xySwap(y,x):
    return np.sin(np.sqrt(x**2 + y**2)) + x/10

# Integrate with .quad() and .dblquad(); it takes ~2 min.
for x_i in range(len(x)):
    for y_j in range(len(y)):

        # Integrate with respect to x, keeping y fixed
        integral_xs[y_j,x_i],_ = spi.quad(lambda x: f_xy(x,y[y_j]), a,x[x_i])

        # Repeat for y, integrating with respect to y, keeping x fixed
        integral_ys[y_j,x_i],_ = spi.quad(lambda y: f_xy(x[x_i],y), a,y[y_j])

        # And now for the double integral
        integral_xys[y_j,x_i],_ = spi.dblquad(f_xySwap, a, x[x_i], lambda x: a, lambda x: y[y_j])


In [ ]:
# %% Exercise 4
#    Continue ...

# Plot
phi = (1 + np.sqrt(5)) / 2
fig,axs = plt.subplots(2,2,figsize=(phi*7,7))

c1 = axs[0,0].contourf(X,Y,function_surf,levels=50,cmap='jet',vmin=-1.5,vmax=1.2)
fig.colorbar(c1, ax=axs[0,0])
axs[0,0].set_title('The function')

c2 = axs[0,1].contourf(X,Y,integral_xys,levels=50,cmap='jet',vmin=-27,vmax=0)
fig.colorbar(c2, ax=axs[0,1])
axs[0,1].set_title(r'$\int\int f(x,y) \,dx\,dy$')

c3 = axs[1,0].contourf(X, Y, integral_xs,levels=50,cmap='jet',vmin=-7.2,vmax=2.2)
fig.colorbar(c3, ax=axs[1,0])
axs[1,0].set_title(r'$\int f(x,y) \,dx$')

c4 = axs[1,1].contourf(X, Y, integral_ys,levels=50,cmap='jet',vmin=-11,vmax=3)
fig.colorbar(c4, ax=axs[1,1])
axs[1,1].set_title(r'$\int f(x,y) \,dy$')

plt.tight_layout()

plt.savefig('fig11_codechallenge_94_exercise_4.png')
plt.show()
files.download('fig11_codechallenge_94_exercise_4.png')


In [ ]:
# %% Exercise 5
#    Draw some cross sections of the function and its integral

# Row index
row = 25

# Plot
phi = (1 + np.sqrt(5)) / 2
plt.figure(figsize=(phi*5,5))

plt.plot(x,function_surf[row,:],lw=3,alpha=.9,label='Function')
plt.plot(x,integral_xs[row,:],'--',lw=3,alpha=.9,label='Integral over x (scipy)')
plt.plot(x,integral_x[row,:],'--',lw=3,alpha=.9,label='Integral over x (numpy)')

plt.xlim(x[[0,-1]])
plt.xlabel('x')
plt.ylabel(r'$f(x,y)$  or  $\int f(x,y)\,dx$')
plt.title(f'Row {row} of the function and its integral (est. with scipy and numpy)')
plt.legend()

plt.savefig('fig12_codechallenge_94_exercise_5.png')
plt.show()
files.download('fig12_codechallenge_94_exercise_5.png')


In [ ]:
# %% Exercise 6
#    The differences in exercise 5 are due to both numerical instability and
#    constant of integration; add a constant of integration to the numpy
#    estimation but not the scipy one; use f(-5,0); then redraw the plot in
#    exercise 5

# See above
